In [5]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1983142975. Use `wandb login --relogin` to force relogin


True

In [5]:
from model.model import Model
import torch
ckpt = "/home/liyongchun/project/experiment/Framework/version_1/logs/THUCNews/hdlukghv/checkpoints/epoch=4-step=3520.ckpt"
checkpoint = torch.load(ckpt)
print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])


In [ ]:
checkpoint["state_dict"]

Extract nn.Module from Lightning checkpoints

In [10]:
from transformers import BertModel, BertTokenizer
from torch import nn
class BertForTHUCNews(nn.Module):

    def __init__(self, ckpt):
        super(BertForTHUCNews, self).__init__()
        self.bert = BertModel.from_pretrained(ckpt)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(768, 10)

    def forward(self, input_ids, attention_mask):
        _, pooled = self.bert(input_ids, attention_mask=attention_mask,
                              return_dict=False)
        out = self.fc(pooled)
        return out
model = BertForTHUCNews("/home/liyongchun/pre_trained_model/bert-base-chinese")
model_weights = checkpoint["state_dict"]
# update keys by dropping `auto_encoder.`
for key in list(model_weights):
    model_weights[key.replace("model.", "")] = model_weights.pop(key)
model.load_state_dict(model_weights)

tokenizer = BertTokenizer.from_pretrained("/home/liyongchun/pre_trained_model/bert-base-chinese")

Some weights of the model checkpoint at /home/liyongchun/pre_trained_model/bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
sentence = "2岁男童爬窗台不慎7楼坠下获救(图)"
encoding = tokenizer(sentence)
input_ids = torch.tensor(encoding.input_ids).unsqueeze(0)
attention_mask = torch.tensor(encoding.attention_mask).unsqueeze(0)
y = model(input_ids, attention_mask)
y

tensor([[-1.8133, -1.1375, -2.5621, -0.9243, -0.1002,  9.3422, -0.1749, -1.3712,
         -0.8469, -1.1613]], grad_fn=<AddmmBackward0>)

In [ ]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
    LoraConfig,
)

# import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

import peft

: 